In [1]:
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
data = open('./data/input.txt','r')
data = list(data)
data_use = []
data_f = []
index = 0
temp = ''
for d in data:
    if d == "<start>\r\n":
        index = 1
        continue
    if d == '<end>\r\n':
        index = 0
    if index == 1:
        temp = temp + str(d)
    if index == 0:
        data_use.append(temp)
        temp = ''
#print(d)
for d in data_use:
    if len(d) > 40:
        data_f.append(d)
data_use = data_f
all_character = set()
counter = 0
for d in data_use:
    for dd in d:
        all_character.add(dd)
all_character = list(all_character)
nletter = len(all_character)

In [3]:
print(data_use[2])

XIV?me si?cle)
Z:Transcrit et/ou corrig? par Michel BELLON - 2005-03-27
Z:Pour toute observation mailto:galouvielle@free.fr
M:2/2
L:1/4
Q:1/4=110
K:Fmaj
V:1
dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G|F4|dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G/2A/4F/4|G2 zG|
w:Stel-__la splen-dens in mon-____te ut so-lis ra-_di-um Mi-__ra-cu-lis ser-ra-____to ex-au-di po-_pu-__lum  Prin-
Gd eg|ef/2d/2 ce|fd c/2B/2B/2A/2|G2 zG|Gd eg|ef/2d/2 ce|fd c/2B/2B/2A/2|G2 z2||
w:ci-pes et ma-gna-__ tes ex-stir-pe re-_gi-_a sae-cu-li po-tes-ta-__tes ob-ten-ta ve-_ni-_a
dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G|F4|dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G/2A/4F/4|G4|]
w:Pe-__cca mi num pro-cla-____mant tu den tes pec-_to-ra Po-__pli te fle-xo cla-____mant hic A-ve Ma-_ri-__a
V:2
G2 GB|dG F2|B2 cd|GF cd|f4| G2 GB|dG F2|B2 cd|GF Bc|d2 zd-|
d G2 G|cB A2|FG Bc|d2 zd-|d G2 G|cB A2|FG Bc|d2 z2|
G2 GB|dG F2|B2 cd|GF cd|f4| G2 GB|dG F2|B2 cd|GF Bc|d4|]



In [4]:
def lineToTensor(line):
    tensor = torch.zeros(len(line),nletter)
    position = np.zeros(len(line))
    for i, letter in enumerate(line):
        position[i] = all_character.index(letter)
        tensor[i][all_character.index(letter)] = 1
    position = torch.LongTensor(position[1:])
    return autograd.Variable(tensor[:-1,:]).cuda(),autograd.Variable(position).cuda()

In [12]:
def batch_g(data,batch_size):
    lyric = np.random.randint(len(data))
    #print(len(data[lyric]))
    if len(data[lyric])-batch_size >0:
        interval_s = np.random.randint(len(data[lyric])-batch_size)
        return data[lyric][interval_s:interval_s+batch_size]
    else:
        return []

In [13]:
class LSTM(nn.Module):
    def __init__(self,letter_dim,hidden_dim):
        super(LSTM, self).__init__()
        self.letter_dim = letter_dim
        self.hidden_dim = hidden_dim
        self.hidden = self.init_hidden()
        
        self.lstm = nn.LSTM(letter_dim,hidden_dim)
        self.hidden2out = nn.Linear(hidden_dim,letter_dim)
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)).cuda(),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)).cuda())
    def forward(self,line):
        lstm_out, self.hidden = self.lstm(line.view(len(line),1,-1), self.hidden)
        out_space = self.hidden2out(lstm_out.view(len(lstm_out),-1))
        out_scores = F.softmax(out_space / 1.0,dim = 1)
        return out_scores

In [14]:
hidden_dim = 100
batch_size = 100
lstm = LSTM(nletter,hidden_dim).cuda()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm.parameters(), lr=0.001)

In [15]:
for epoch in range(1000000):
    lstm.zero_grad()
    batch = batch_g(data_use,batch_size)
#     print(batch)
#     print('\n')
    if len(batch)>0:
        batch_in,batch_t = lineToTensor(batch)
        lstm.hidden = lstm.init_hidden()
        out_scores = lstm(batch_in)
        loss = loss_function(out_scores,batch_t)
        if epoch % 10000 == 0:
            print(loss)
        loss.backward()
        optimizer.step()

Variable containing:
 4.5434
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.3015
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.2514
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.1084
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.0640
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.0433
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.1144
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.2888
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.0564
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.0651
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.3035
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.0086
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.0535
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.2270
[torch.cuda.FloatTenso

In [17]:
print(out_scores.data.cpu().numpy()[1])

[  9.99993801e-01   3.47041641e-17   4.93132755e-17   3.68659825e-15
   3.06322749e-17   8.04442989e-18   3.88795700e-17   3.20973308e-17
   4.11795291e-17   3.63599381e-17   1.22431499e-17   3.43132742e-20
   8.23901034e-23   3.28833212e-17   2.95029715e-17   6.20019910e-06
   1.47898295e-12   7.24900540e-09   4.96574463e-15   1.15888202e-13
   8.52889810e-16   3.56669142e-15   3.39689792e-17   4.13975078e-17
   3.05936224e-17   1.07577800e-17   2.34670561e-17   3.56275756e-17
   3.33338236e-17   1.33306230e-19   6.14014747e-20   5.85424850e-30
   3.14274337e-17   2.93518721e-17   3.64735762e-17   2.84013158e-17
   3.18630965e-17   4.43852676e-17   1.14399286e-19   3.21241547e-17
   6.77188598e-20   1.68442933e-11   3.22934587e-17   3.06237450e-17
   8.51846337e-23   3.20602681e-12   4.02311268e-17   3.29599280e-17
   3.47560978e-17   2.45767817e-15   3.27668674e-17   8.23728844e-29
   3.07898428e-17   1.18559730e-24   1.45255258e-16   3.27976319e-17
   1.52501862e-14   4.32249714e-17

In [18]:
present = torch.zeros(1,nletter)
present[0][all_character.index('>')] = 1
present = autograd.Variable(present).cuda()
lyric = []
lyric.append(all_character[int(torch.distributions.Categorical(lstm(present).data.cpu()).sample().numpy())])
for num_generation in range(400):
    #print(lstm.forward(present).cpu())
    next_char_index = int(torch.distributions.Categorical(lstm(present).data.cpu()).sample().numpy())
    next_char = all_character[next_char_index]
    lyric.append(next_char)
    present = torch.zeros(1,nletter)
    present[0][all_character.index(next_char)] = 1
    present = autograd.Variable(present).cuda()

In [19]:
print(str(lyric))

['f', 'e', ' ', 'd', 'B', 'A', 'G', '|', '\r', '\n', 'A', 'G', 'A', 'B', ' ', 'A', 'G', 'A', 'B', '|', 'G', 'A', 'B', 'd', ' ', 'd', 'B', 'A', 'G', '|', 'A', 'G', 'A', 'B', ' ', 'A', '2', 'G', 'A', '|', 'B', '2', 'A', 'B', ' ', 'A', 'G', 'E', 'G', '|', '\r', '\n', 'A', '2', 'A', 'B', ' ', 'A', 'G', 'E', 'G', '|', 'A', 'B', 'A', 'G', ' ', 'A', 'B', 'A', 'G', '|', 'A', 'B', 'A', 'G', ' ', 'A', 'B', 'A', 'G', '|', '\r', '\n', 'A', 'G', 'A', 'B', ' ', 'A', 'G', 'A', 'B', '|', 'c', 'A', 'G', 'E', ' ', 'A', 'B', 'd', 'e', '|', '|', '\r', '\n', '|', ':', ' ', 'f', 'e', 'd', 'e', ' ', 'd', 'B', 'A', 'G', '|', 'c', 'B', 'A', 'G', ' ', 'A', 'B', 'A', 'G', '|', '\r', '\n', 'A', 'G', 'A', 'B', ' ', 'A', 'G', 'A', 'B', '|', 'G', 'A', 'B', 'd', ' ', 'd', 'B', 'd', 'e', '|', 'f', 'd', 'e', 'd', ' ', 'B', 'A', 'G', 'E', '|', '\r', '\n', 'G', 'A', 'B', 'd', ' ', 'd', '2', 'B', 'd', '|', 'e', 'd', 'e', 'f', ' ', 'e', 'd', 'B', 'd', '|', 'e', '2', 'f', 'e', ' ', 'd', 'B', 'd', 'e', '|', 'f', 'd', 'd', 'e

In [20]:
Music = ""
for char in lyric:
    Music = Music + char
print(Music)

fe dBAG|
AGAB AGAB|GABd dBAG|AGAB A2GA|B2AB AGEG|
A2AB AGEG|ABAG ABAG|ABAG ABAG|
AGAB AGAB|cAGE ABde||
|: fede dBAG|cBAG ABAG|
AGAB AGAB|GABd dBde|fded BAGE|
GABd d2Bd|edef edBd|e2fe dBde|fdde fded|
BABd edBd|edef edBd|e2fe d2Bd|
e2fe defd|edef edBd|efed BAGA|BAGE G2EG|
AGAB AGEG|ABAG AGEG|ABAG A2GA|
BAGB AGEG|ABAG AGEG|
ABAG ABAG|ABAG ABAG|ABAG ABAG|
AGAB AGAB|cAGE ABde|fded Bded|BAGE 
